# Materials

PyNE `Material` objects provide a way of representing, manipulating, and storing materials. A `Material` object is a collection of nuclides with various mass fractions (though methods for converting to/from atom fractions are present as well). Optionally, a `Material` object may have an associated mass. By keeping the mass and the composition separate, operations that only affect one attribute may be performed independent of the other. Most of the functionality of the `Material` class is
implemented in a C++, so this interface is very fast and light-weight.

A `Material` may be initialized in a number of different ways.  For example, initializing from
dictionaries of compositions are shown below. First import the `Material` class:

In [567]:
from pyne.material import Material

Now create a low enriched uranium (leu) with a mass of 42:

In [568]:
# Low enriched uranium with mass 42
leu = Material({'U238': 0.96, 'U235': 0.04}, 42)
print(leu)

Material:
mass = 42.0
density = -1.0
atoms per molecule = -1.0
-------------------------
U235   0.04
U238   0.96


Create another `Material`, this one with 9 components. Notice that the mass is 9 x 1.0 = 9.0: 

In [569]:
# Material with 9 components
nucvec = {10010:  1.0, 80160:  1.0, 691690: 1.0, 922350: 1.0,
          922380: 1.0, 942390: 1.0, 942410: 1.0, 952420: 1.0,
          962440: 1.0}
mat = Material(nucvec)
print(mat)

Material:
mass = 9.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H1     0.1111111111111111
O16    0.1111111111111111
Tm169  0.1111111111111111
U235   0.1111111111111111
U238   0.1111111111111111
Pu239  0.1111111111111111
Pu241  0.1111111111111111
Am242  0.1111111111111111
Cm244  0.1111111111111111


Materials may also be initialized from plain text or HDF5 files (see ``Material.from_text()`` and
``Material.from_hdf5()``).

------

## Normalization

When creating a new instance of the `Material` class, the mass fractions that define it are already normalized. However, you can retrieve the unnormalized mass vector using the ``Material.mult_by_mass()`` method. If you need to normalize the mass or composition, you can use the provided normalization routines: ``Material.normalize()`` or ``Material.norm_comp()``.

For example, let's consider a scenario where we have 42 units of low-enriched uranium (LEU), consisting of 1.68 units of U-235 and 40.32 units of U-238.

In [570]:
leu.mult_by_mass()

{922350000: 1.68, 922380000: 40.32}

Recall that `mat` has a mass of 9. Here it is normalized to a mass of 1:

In [571]:
# Normalize
mat.normalize()
print(mat)

Material:
mass = 1.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H1     0.1111111111111111
O16    0.1111111111111111
Tm169  0.1111111111111111
U235   0.1111111111111111
U238   0.1111111111111111
Pu239  0.1111111111111111
Pu241  0.1111111111111111
Am242  0.1111111111111111
Cm244  0.1111111111111111


In [572]:
# Check mass
mat.mass

1.0

-----------

## Material Arithmetic

The `Material` class also provides support for arithmetic operations with numeric types. When you add two instances of `Material` together, a new `Material` object is returned. The values of this new object are the weighted union of the two original materials.

On the other hand, multiplying a `Material` by a numeric type, such as 2, will result in the mass of the original material being doubled. The composition of the material remains unchanged.

In [573]:
# Multiply by 2
other_mat = mat * 2
print(other_mat)

Material:
mass = 2.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H1     0.11111111111111108
O16    0.11111111111111108
Tm169  0.11111111111111108
U235   0.11111111111111108
U238   0.11111111111111108
Pu239  0.11111111111111108
Pu241  0.11111111111111108
Am242  0.11111111111111108
Cm244  0.11111111111111108


In [574]:
# Mass
other_mat.mass

2.0

In [575]:
# Add leu to mat multiply by 18
weird_mat = leu + mat * 18
print(weird_mat)


Material:
mass = 60.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H1     0.03333333333333332
O16    0.03333333333333332
Tm169  0.03333333333333332
U235   0.06133333333333332
U238   0.7053333333333334
Pu239  0.03333333333333332
Pu241  0.03333333333333332
Am242  0.03333333333333332
Cm244  0.03333333333333332


**Note:** There are also ways of mixing `Materials` by volume using known densities. See the `pyne.MultiMaterial` class for more information.

---------------

## Raw Member Access

You may also change the attributes of a material directly without generating a new 
material instance.

In [576]:
# Update mass
other_mat.mass = 10

# Update material composition
other_mat.comp = {10020000: 3, 922350000: 15.0}

print(other_mat)

Material:
mass = 10.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H2     3.0
U235   15.0


Of course when you do this you have to be careful because the composition and mass may now be out
of sync.  This may always be fixed with normalization.

In [577]:
# Normalize composition
other_mat.norm_comp()
print(other_mat)

Material:
mass = 10.0
density = -1.0
atoms per molecule = -1.0
-------------------------
H2     0.16666666666666666
U235   0.8333333333333334


--------

## Indexing & Slicing

In addition, the `Material` class provides powerful indexing and slicing capabilities for accessing and manipulating sub-materials within the material or its composition.

- When indexing into the composition, you can only use integer keys to retrieve the normalized value of a specific component. This allows you to retrieve the composition fraction of a specific nuclide.

- When indexing into the material, you have more flexibility and can perform a wide range of operations. It returns the unnormalized mass weight of the specified sub-material. You can use integer keys, string keys, slices, or sequences of nuclides to index into the material and perform operations on specific parts of the material.

In [578]:
# Check composition of low enriched uranium
leu.comp[922350000]

0.04

In [579]:
# Check unnormalized mass of U235
leu['U235']

1.68

In [580]:
# Slice from U to Pu
print(weird_mat['U':'Am'])

Material:
mass = 50.0
density = -1.0
atoms per molecule = -1.0
-------------------------
U235   0.07359999999999998
U238   0.8464
Pu239  0.03999999999999998
Pu241  0.03999999999999998


In [581]:
# Setting mass for H2 42
other_mat[:920000000] = 42
print(other_mat)

Material:
mass = 50.333333333333336
density = -1.0
atoms per molecule = -1.0
-------------------------
H2     0.8344370860927152
U235   0.16556291390728478


In [582]:
# Remove specific element
del mat[962440, 'TM169', 'Zr90', 80160]
print(mat)

Material:
mass = 0.6666666666666667
density = -1.0
atoms per molecule = -1.0
-------------------------
H1     0.16666666666666663
U235   0.16666666666666663
U238   0.16666666666666663
Pu239  0.16666666666666663
Pu241  0.16666666666666663
Am242  0.16666666666666663


Other methods also exist for obtaining commonly used sub-materials, such as gathering the Uranium or 
Plutonium vector.  

### Molecular Weights & Atom Fractions

You may also calculate the molecular weight of a material via the ``Material.molecular_weight`` method.
This uses the ``pyne.data.atomic_mass`` function to look up the atomic mass values of
the constituent nuclides.

In [583]:
# Check molecule mass of low enriched uranium
leu.molecular_mass()

237.9290363047951

Please note that the default assumption is that materials have one atom per molecule, which may not be accurate for more complex substances. For instance, water consists of multiple atoms per molecule. If the number of atoms per molecule is not specified, the calculation of molecular weight will be incorrect by a factor of 3. To address this, it is necessary to provide the accurate number when using the method. If no valid number of molecules is stored for the material, this will assign the correct attribute to the class.

In [584]:
# New H2O material
h2o = Material({'H1': 0.11191487328808077, 'O16': 0.8880851267119192})

# Check molecule mass of H2O
h2o.molecular_mass()

6.003521561386799

In [585]:
# Set molecule mass of H2O to 3
h2o.molecular_mass(3.0)
print(h2o)

Material:
mass = 1.0
density = -1.0
atoms per molecule = 3.0
------------------------
H1     0.11191487328808077
O16    0.8880851267119192


It is valuable to have the capability to convert the current mass-weighted material into an atom fraction mapping using the `Material.to_atom_frac()` method. For instance, in the case of water, when the number of atoms per molecule is accurately specified, the atom fraction returned will be normalized accordingly. On the other hand, if the atoms per molecule are set to their default value in the class, a fractional number of atoms will be returned. This functionality allows for flexible and precise representation of atom fractions in the material.

In [586]:
h2o.to_atom_frac()

{10010000: 1.9999999999946356, 80160000: 1.0000000000053646}

In [587]:
# Set number of atoms per molecule of H2O to -1
h2o.atoms_per_molecule = -1.0
h2o.to_atom_frac()

{10010000: 0.6666666666648785, 80160000: 0.3333333333351215}

Moreover, it may be necessary to convert a given set of atom fractions into a new material stream. This can be achieved using the `Material.from_atom_frac()` method, which will clear the existing composition of the material and replace it with the mass-weighted values based on the atom fractions provided. It is important to note that when initializing a material from atom fractions, the sum of all atom fractions will be stored as the atoms per molecule attribute in this class. Additionally, if a mass is not already specified for the material, the molecular weight will be utilized for the conversion process.

In [588]:
h2o_atoms = {10010000: 2.0, 'O16': 1.0}
h2o = Material()
h2o.from_atom_frac(h2o_atoms)

print(h2o.comp)
print(h2o)

{10010000: 0.11191487328888054, 80160000: 0.8880851267111195}
Material:
mass = 18.01056468408
density = -1.0
atoms per molecule = 3.0
------------------------
H1     0.11191487328888054
O16    0.8880851267111195


Furthermore, it is also possible to create a new material from atom fractions using other materials as references. This scenario often occurs in reactors, where the fuel vector is combined within another chemical form. Here is an example of obtaining a uranium oxide material by specifying the atom fractions of oxygen and low-enriched uranium:

In [589]:
# New UO2 material
uox = Material()

# Add low enriched uranium as reference with O16
uox.from_atom_frac({leu: 1.0, 'O16': 2.0})

print(uox)

Material:
mass = 269.9188655439951
density = -1.0
atoms per molecule = 3.0
------------------------
O16    0.11851646299241672
U235   0.03525934148030333
U238   0.84622419552728


**NOTE:** Materials may be used as keys in a dictionary because they are hashable.

### User-defined Metadata

Another important feature of materials is the availability of a `metadata` attribute, which allows users to store customized information about the material. This attribute serves as an in-memory JSON object attached to the underlying C++ class. As a result, the content that can be stored in the `metadata` is subject to the same limitations as JSON. Typically, the top-level of the `metadata` is expected to be a dictionary, although this requirement is not strictly enforced. Users can utilize the `metadata` attribute to store various details such as units, comments, provenance information, or any other relevant information they deem necessary for their specific use case.

In [590]:
# Add metadata units
leu = Material({922350: 0.05, 922380: 0.95}, 15, metadata={'units': 'kg'})
leu

pyne.material.Material({922350000: 0.05, 922380000: 0.95}, 15.0, -1.0, -1.0, {"units":"kg"})

In [591]:
print(leu)

Material:
mass = 15.0
density = -1.0
atoms per molecule = -1.0
units = kg
-------------------------
U235   0.05
U238   0.95


In [592]:
leu.metadata

{"units":"kg"}

In [593]:
# Add more metadata
m = leu.metadata
m['comments'] = ['Anthony made this material.']
leu.metadata['comments'].append('And then Katy made it better!')
m['id'] = 42
leu.metadata

{"comments":["Anthony made this material.","And then Katy made it better!"],"id":42,"units":"kg"}

In [594]:
# Update a specific metadata
leu.metadata = {'units': 'solar mass'}
leu.metadata

{"units":"solar mass"}

This also manipulate other variables.

In [595]:
m

{"units":"solar mass"}

In [596]:
leu.metadata['units'] = 'not solar masses'
leu.metadata['units']

'not solar masses'

As demonstrated above, the attrs interface allows direct access to the underlying JSON object, enabling users to manipulate it directly or by assigning it to another variable. Furthermore, it is possible to replace the `metadata` object with a new object of the appropriate type. It's important to note that when the `metadata` object is replaced, any previous views or references to the old object become invalid. Therefore, care should be taken when modifying or replacing the `metadata` object to avoid unintended consequences.